In [1]:
import numpy as np
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
from tqdm.auto import tqdm
import pandas as pd
tqdm.pandas()
import string
import re
import gensim
from collections import Counter
import pickle
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, Embedding, SpatialDropout1D, concatenate
from tensorflow.keras.layers import GRU, LSTM,Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Lambda
import warnings
import gc
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
from unidecode import unidecode
import time
np.random.seed(42)
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("../data/train_ft.csv")
train_aug=pd.read_csv("../data/itrain_aug.csv") # Augmentation ...
train=train.append(train_aug[train_aug["Sub1_Toxic"]==1].sample(300),ignore_index=True)
train.drop_duplicates('text',inplace=True)
train.reset_index(drop=True, inplace=True)
test=pd.read_csv("../data/test_ft.csv")
FEATURES=['readability','!','?',"caps_vs_length","Partizip II","Präteritum_ich","punc","error",'Präsens_ich',"present","future",'words_vs_unique',"pos","neg","num_urls","mod","emoji","certainity","uncertainity","num_words"]
features = train[FEATURES].fillna(0)
test_features = test[FEATURES].fillna(0)
ss = StandardScaler()
ss.fit(np.vstack((features, test_features)))
features = ss.transform(features)
test_features = ss.transform(test_features)
y_train = train[["Sub1_Toxic","Sub2_Engaging","Sub3_FactClaiming"]].values
X_train = train['text'].fillna("etwas").values
X_test = test['text'].fillna("etwas").values

In [3]:
max_features = 17194
maxlen = 1000
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train_sequence = tokenizer.texts_to_sequences(X_train)
X_test_sequence = tokenizer.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(X_train_sequence, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test_sequence, maxlen=maxlen)
print(len(tokenizer.word_index))

17194


In [4]:
EMBEDDING_FILE_FASTTEXT="../data/cc.de.300.vec"
EMBEDDING_FILE_TWITTER="../data/glove.txt"
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index_ft = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in tqdm(open(EMBEDDING_FILE_FASTTEXT,encoding='utf-8')))
embeddings_index_tw = dict(get_coefs(*o.strip().split()) for o in tqdm(open(EMBEDDING_FILE_TWITTER,encoding='utf-8')))
spell_model = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE_FASTTEXT)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [5]:
# This code is  based on: Spellchecker using Word2vec by CPMP
# https://www.kaggle.com/cpmpml/spell-checker-using-word2vec

words = list(spell_model.index_to_key)

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

# Use fast text as vocabulary
def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyzäöüß' # Add german
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def singlify(word):
    return "".join([letter for i,letter in enumerate(word) if i == 0 or letter != word[i-1]])

In [6]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words,600))

something_tw = embeddings_index_tw.get("etwas")
something_ft = embeddings_index_ft.get("etwas")

something = np.zeros((600,))
something[:300,] = something_ft
something[300:600,] = something_tw

def embed_word(embedding_matrix,i,word):
    embedding_vector_ft = embeddings_index_ft.get(word)
    if embedding_vector_ft is not None: 
        embedding_matrix[i,:300] = embedding_vector_ft
        embedding_vector_tw = embeddings_index_tw.get(word)
        if embedding_vector_tw is not None:
            embedding_matrix[i,300:600] = embedding_vector_tw

            
for word, i in tqdm(word_index.items()):
    
    if i >= max_features: continue
        
    if embeddings_index_ft.get(word) is not None:
        embed_word(embedding_matrix,i,word)
    else:
        if len(word) > 20:
            embedding_matrix[i] = something
        else:
            word2 = correction(word)
            if embeddings_index_ft.get(word2) is not None:
                embed_word(embedding_matrix,i,word2)
            else:
                word2 = correction(singlify(word))
                if embeddings_index_ft.get(word2) is not None:
                    embed_word(embedding_matrix,i,word2)
                else:
                    embedding_matrix[i] = something

  0%|          | 0/14036 [00:00<?, ?it/s]

In [7]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.max_score = 0
        self.not_better_count = 0

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            if (score > self.max_score):
                print("*** New High Score (previous: %.6f) \n" % self.max_score)
                model.save_weights("../checkpoints/f_weights.h5")
                self.max_score=score
                self.not_better_count = 0
            else:
                self.not_better_count += 1
                if self.not_better_count > 3:
                    print("Epoch %05d: early stopping, high score = %.6f" % (epoch,self.max_score))
                    self.model.stop_training = True

In [8]:
def get_model(features,clipvalue=1.,num_filters=40,dropout=0.5,embed_size=600):
    features_input = Input(shape=(features.shape[1],))
    inp = Input(shape=(maxlen, ))
    
    # Layer 1: concatenated fasttext and glove twitter embeddings.
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)

    # Layer 2: SpatialDropout1D(0.5)
    x = SpatialDropout1D(dropout)(x)
    
    # Layer 3: Bidirectional CuDNNLSTM
    x = Bidirectional(LSTM(num_filters, return_sequences=True))(x)


    # Layer 4: Bidirectional CuDNNGRU
    x, x_h, x_c = Bidirectional(GRU(num_filters, return_sequences=True, return_state = True))(x)  
    
    # Layer 5: A concatenation of the last state, maximum pool, average pool and 
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    x = concatenate([avg_pool, x_h, max_pool,features_input])
    
    # Layer 6: output dense layer.
    outp = Dense(3, activation="sigmoid")(x)

    model = Model(inputs=[inp,features_input], outputs=outp)
    import tensorflow as tf
    adam = tf.optimizers.Adam(clipvalue=clipvalue)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

In [ ]:
del spell_model
del embeddings_index_ft
del embeddings_index_tw

model = get_model(features)

batch_size = 32


epochs = 100
gc.collect()
K.clear_session()

num_folds = 10 

predict = np.zeros((test.shape[0],3))

kf = KFold(n_splits=num_folds, shuffle=True, random_state=239)

for train_index, test_index in kf.split(x_train):
    
    kfold_y_train,kfold_y_test = y_train[train_index], y_train[test_index]
    kfold_X_train = x_train[train_index]
    kfold_X_features = features[train_index]
    kfold_X_valid = x_train[test_index]
    kfold_X_valid_features = features[test_index] 
    
    gc.collect()
    K.clear_session()
    
    model = get_model(features)
    
    ra_val = RocAucEvaluation(validation_data=([kfold_X_valid,kfold_X_valid_features], kfold_y_test), interval = 1)
    
    model.fit([kfold_X_train,kfold_X_features], kfold_y_train, batch_size=batch_size, epochs=epochs, verbose=1,
             callbacks = [ra_val])
    gc.collect()
    model.load_weights("../checkpoints/f_weights.h5")
    
    predict += model.predict([x_test,test_features], batch_size=batch_size,verbose=1) / num_folds

print("Done")

Epoch 1/100
10/10 [==============================] - 3s 226ms/step

 ROC-AUC - epoch: 1 - score: 0.693396 

*** New High Score (previous: 0.000000) 

Epoch 2/100
10/10 [==============================] - 2s 222ms/step

 ROC-AUC - epoch: 2 - score: 0.727122 

*** New High Score (previous: 0.693396) 

Epoch 3/100
10/10 [==============================] - 2s 214ms/step

 ROC-AUC - epoch: 3 - score: 0.753937 

*** New High Score (previous: 0.727122) 

Epoch 4/100
10/10 [==============================] - 2s 218ms/step

 ROC-AUC - epoch: 4 - score: 0.764445 

*** New High Score (previous: 0.753937) 

Epoch 5/100
10/10 [==============================] - 2s 230ms/step

 ROC-AUC - epoch: 5 - score: 0.765025 

*** New High Score (previous: 0.764445) 

Epoch 6/100
10/10 [==============================] - 2s 222ms/step

 ROC-AUC - epoch: 6 - score: 0.778086 

*** New High Score (previous: 0.765025) 

Epoch 7/100
10/10 [==============================] - 2s 224ms/step

 ROC-AUC - epoch: 7 - score: 0.7

In [ ]:
test[["1","2","3"]]=predict
test[["1","2","3"]].to_csv("../submissions/gru-fasttext-p.csv")
test[["1","2","3"]]=predict.round(0).astype(int)
test[["1","2","3"]].to_csv("../submissions/gru-fasttext.csv")